In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns

In [16]:
year = 2022
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

In [17]:
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

In [18]:
soup.findAll('tr',limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]

In [19]:
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
stats = pd.DataFrame(player_stats,columns = headers)

In [20]:
rowindex = ['Age','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
age, G, GS, MP, FG, FGA, FGper, s3P, s3PA, s3Pper, s2P, s2PA, s2Pper, eFGper, FT, FTA, FTper, ORB, DRB,TRB, AST, STL, BLK, TOV,PF,PTS = 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB','AST','STL','BLK','TOV','PF','PTS'

In [21]:
for i in rowindex:
    stats[i] = stats[i].apply(pd.to_numeric)

In [22]:
stats.fillna(value=0)    

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,22.0,TOR,13.0,11.0,24.2,3.1,8.8,0.348,...,0.538,2.1,6.6,8.7,1.5,0.6,0.8,1.2,2.3,7.5
1,Steven Adams,C,28.0,MEM,13.0,13.0,25.9,3.4,6.4,0.530,...,0.706,4.1,4.8,8.8,2.6,0.8,0.8,1.4,1.7,8.6
2,Bam Adebayo,C,24.0,MIA,12.0,12.0,33.1,7.0,13.9,0.503,...,0.840,2.4,8.8,11.2,2.7,1.4,0.5,3.0,3.1,19.7
3,Santi Aldama,PF,21.0,MEM,4.0,0.0,6.8,1.3,4.3,0.294,...,0.667,0.8,0.8,1.5,0.3,0.0,0.0,0.0,0.8,3.3
4,LaMarcus Aldridge,C,36.0,BRK,13.0,0.0,20.5,5.2,8.9,0.586,...,0.750,1.4,4.3,5.7,0.9,0.3,1.2,0.8,1.8,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,Thaddeus Young,PF,33.0,SAS,10.0,0.0,19.1,4.1,6.6,0.621,...,0.385,2.0,2.5,4.5,3.5,1.2,0.6,1.4,1.8,8.7
472,Trae Young,PG,23.0,ATL,14.0,14.0,34.4,9.1,20.2,0.452,...,0.907,0.6,3.1,3.7,9.3,0.9,0.1,3.9,1.6,25.7
473,Omer Yurtseven,C,23.0,MIA,8.0,0.0,3.4,1.0,1.6,0.615,...,0.500,0.3,0.8,1.0,0.0,0.0,0.0,0.1,0.0,2.1
474,Cody Zeller,C,29.0,POR,14.0,0.0,14.1,1.9,3.1,0.614,...,0.730,2.0,2.7,4.7,0.9,0.4,0.1,0.9,2.4,5.8


In [23]:
#The Hollinger Game Score stat: PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV.
hollpts,hollfg,hollfga,hollfta,hollft,hollorb,holldrb,hollstl,hollast,hollblk,hollpf,holltov = [],[],[],[],[],[],[],[],[],[],[],[]

def get_holl(pts,fg,fga,fta,ft,orb,drb,stl,ast,blk,pf,tov):
    hollingerEQ = lambda PTS,FG,FGA,FTA,FT,ORB,DRB,STL,AST,BLK,PF,TOV: PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV
    return hollingerEQ(pts,fg,fga,fta,ft,orb,drb,stl,ast,blk,pf,tov)
    # for i in range(677):
    #     hollpts.append(stats[PTS].loc[i])



In [24]:
GameScore = []
for i in range(len(stats)):
    hollpts.append(stats['PTS'].iloc[i])
    hollfg.append(stats['FG'].iloc[i])
    hollfga.append(stats['FGA'].iloc[i])
    hollfta.append(stats['FTA'].iloc[i])
    hollft.append(stats['FT'].iloc[i])
    hollorb.append(stats['ORB'].iloc[i])
    holldrb.append(stats['DRB'].iloc[i])
    hollstl.append(stats['STL'].iloc[i])
    hollast.append(stats['AST'].iloc[i])
    hollblk.append(stats['BLK'].iloc[i])
    hollpf.append(stats['PF'].iloc[i])
    holltov.append(stats['TOV'].iloc[i])

for i in range(len(stats)):
    GameScore.append(round(get_holl(hollpts[i],hollfg[i],hollfga[i],hollfta[i],hollft[i],hollorb[i],holldrb[i],hollstl[i],hollast[i],hollblk[i],hollpf[i],holltov[i]),5))

In [25]:
stats['Game Score'] = GameScore

In [26]:
stats.head(1)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Game Score
0,Precious Achiuwa,C,22.0,TOR,13.0,11.0,24.2,3.1,8.8,0.348,...,2.1,6.6,8.7,1.5,0.6,0.8,1.2,2.3,7.5,5.76


In [27]:
stats.set_index('Player',inplace=True)

In [29]:
stats.nlargest(10,['Game Score'])

,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Game Score
Player,,,,,,,,,,,,,,,,,,,,,
Nikola Jokić,C,26.0,DEN,12.0,12.0,31.8,9.9,16.7,0.595,1.7,...,2.8,10.8,13.7,6.3,1.4,0.8,3.4,3.1,25.3,24.02
Kevin Durant,PF,33.0,BRK,14.0,14.0,34.6,11.0,18.8,0.586,2.0,...,0.4,7.9,8.4,5.1,0.6,0.6,3.2,1.2,29.6,23.96
Stephen Curry,PG,33.0,GSW,13.0,13.0,33.8,9.0,20.4,0.442,5.2,...,0.8,5.5,6.2,6.7,1.7,0.7,3.3,1.5,28.1,22.49
Giannis Antetokounmpo,PF,27.0,MIL,13.0,13.0,32.9,9.5,19.3,0.490,1.3,...,1.8,9.5,11.3,6.0,1.1,1.7,3.0,3.2,26.5,21.91
Jimmy Butler,SF,32.0,MIA,11.0,11.0,33.0,8.0,15.2,0.527,0.5,...,1.7,3.7,5.5,5.1,2.1,0.4,1.9,2.0,23.6,21.19
Anthony Davis,C,28.0,LAL,14.0,14.0,34.9,9.8,19.4,0.506,0.4,...,3.8,7.5,11.3,3.0,1.3,2.2,2.3,2.2,24.5,20.91
Ja Morant,PG,22.0,MEM,13.0,13.0,35.2,9.8,20.2,0.487,1.7,...,1.3,5.2,6.5,7.4,1.7,0.2,3.8,1.4,26.2,20.55
DeMar DeRozan,SF,32.0,CHI,13.0,13.0,35.3,9.2,18.5,0.496,0.8,...,0.8,4.8,5.6,4.0,0.8,0.4,1.8,2.3,26.1,19.55
Trae Young,PG,23.0,ATL,14.0,14.0,34.4,9.1,20.2,0.452,2.6,...,0.6,3.1,3.7,9.3,0.9,0.1,3.9,1.6,25.7,19.29


In [66]:
teams = []
for i in stats['Tm']:
    if i not in teams and i != None:
        teams.append(i)


In [67]:
teams.sort()

In [68]:
teams

['ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'TOT',
 'UTA',
 'WAS']